In [10]:
from pathlib import Path
import os

In [20]:
dataPath = "F:/project/navigationParadigm/MorrisMaze/data/"
dataFolder = ["ifADo Data","ifADo_2ndRound"]
saveDataPath = Path('F:/project/navigationParadigm/MorrisMaze/analyzeBehData/')
dataPath_exl = Path('F:/project/navigationParadigm/MorrisMaze/data/')
exlName = ["Demographics_04092020_IFaDo.xlsx","Demographic_data_v2.xlsx"]

In [21]:
print(dataPath)

F:/project/navigationParadigm/MorrisMaze/data/


In [23]:
for x in range(len(dataFolder)):
    mypath = dataPath+dataFolder[x]
    print(mypath)
    files = [f for f in os.listdir(dataPath+dataFolder[x]) if os.path.isdir(f)]
    print(files)

F:/project/navigationParadigm/MorrisMaze/data/ifADo Data


FileNotFoundError: [Errno 2] No such file or directory: 'F:/project/navigationParadigm/MorrisMaze/data/ifADo Data'